In [61]:
import polars as pl
import boto3
from botocore.config import Config
from dotenv import load_dotenv
import os
from concurrent.futures import ThreadPoolExecutor

In [10]:
load_dotenv()
bucket_name = 'sagemaker-us-west-2-730335331132'

In [11]:
s3 = boto3.client('s3')

In [19]:
images = pl.read_csv('new_classes.csv')

In [56]:
def download_class_split(old_class, new_class, split):
    if split == 'test':
        path = 'Food2k_complete/'
    else:
        path = f'Food2k_complete/{split}/'
    download_path = f'data/{split}/'
    result = s3.list_objects(Bucket=bucket_name, Prefix=f'{path}{old_class}/', Delimiter='/')
    print(f'Downloading {len(result["Contents"])} images')
    for o in result['Contents']:
        filename = o['Key'].split('/')[-1]
        os.makedirs(f'{download_path}{new_class}/', exist_ok=True)
        s3.download_file(bucket_name, o['Key'], f'{download_path}{new_class}/{filename}')

In [57]:
def download_class(old_class, new_class):
    download_class_split(old_class, new_class, 'train')
    download_class_split(old_class, new_class, 'val')
    download_class_split(old_class, new_class, 'test')

In [62]:
with ThreadPoolExecutor(max_workers=8) as executor:
    for i, row in enumerate(images.rows(named=True)):
        print(f'Downloading class {i}, {row["name"]}')
        executor.submit(download_class, row['class'], row['new_class'])